In [ ]:
import random
import networkx as nx # type: ignore
import colorsys
from itertools import product

%run city_utilities.ipynb

In [ ]:
def find_best_grid_dimensions(width, height, num_horiz_roads, num_vert_roads):
    """Finds the best grid spacing so roads are evenly spaced and fit within the area."""
    horiz_spacing = int(width / (num_horiz_roads - 1))
    vert_spacing = int(height / (num_vert_roads - 1))

    # Define the search space for small adjustments
    variables = list(
        product(
            range(width - 5, width + 6),
            range(height - 5, height + 6),
            range(num_horiz_roads - 5, num_horiz_roads + 6),
            range(num_vert_roads - 5, num_vert_roads + 6),
            range(horiz_spacing - 5, horiz_spacing + 6),
            range(vert_spacing - 5, vert_spacing + 6),
        )
    )

    def error_function(variables):
        width, height, num_horiz_roads, num_vert_roads, horiz_spacing, vert_spacing = variables
        horiz_error = abs(width - (num_horiz_roads - 1) * horiz_spacing) / width
        vert_error = abs(height - (num_vert_roads - 1) * vert_spacing) / height
        
        # Disqualify if total width or height taken up by roads spills over
        if (num_horiz_roads - 1) * horiz_spacing > width or (num_vert_roads - 1) * vert_spacing > height:
            return float('inf')
        
        return horiz_error + vert_error

    return min(variables, key=error_function)


In [2]:
def generate_zone_labels(num_zones):
    # Generate random colours
    colours = []
    transparency = 0.5
    for i in range(num_zones):
        hue = i / num_zones
        lightness = 0.75
        saturation = 0.9
        rgb = colorsys.hls_to_rgb(hue, lightness, saturation)
        hex_colour = "#{:02x}{:02x}{:02x}{:02x}".format(
            int(rgb[0] * 255), int(rgb[1] * 255), int(rgb[2] * 255), int(transparency * 255)
        )
        colours.append(hex_colour)
    random.shuffle(colours)

    # Generate random names
    prefixes = [
        "North",
        "South",
        "East",
        "West",
        "Central",
        "Lower",
        "Upper",
        "Old",
        "New",
        "Inner",
        "Outer",
        "Upper",
        "Lower",
        "Little",
        "Big",
        "Saint",
        "King",
        "Queen",
        "Prince",
        "Princess",
        "Royal",
        "Grand",
    ]
    suffixes = [
        "District",
        "Quarter",
        "Heights",
        "Village",
        "Park",
        "Square",
        "Gardens",
        "Town",
        "City",
        "Vista",
        "Valley",
        "Hills",
        "Meadows",
        "Forest",
        "Grove",
        "Lake",
        "River",
        "Bay",
        "Harbor",
        "Port",
        "Beach",
        "Cove",
    ]

    names = [f"{prefix} {suffix}" for prefix, suffix in product(prefixes, suffixes)]
    random.shuffle(names)
    names = names[: num_zones]
    
    return names, colours

In [3]:
def divide_and_assign_zone_data(width, height, num_zones, names, colours):
    min_size = min(width, height) // (num_zones // 2)

    def split_area(x, y, width, height, remaining_zones):
        if remaining_zones <= 1 or min(width, height) < min_size:
            return [{"x": x, "y": y, "width": width, "height": height}]

        split_vertically = width > height
        split_range = (width if split_vertically else height) // 3
        split_pos = random.randint(split_range, 2 * split_range)

        first_half = remaining_zones // 2
        second_half = remaining_zones - first_half

        if split_vertically:
            return split_area(x, y, split_pos, height, first_half) + split_area(
                x + split_pos, y, width - split_pos, height, second_half
            )
        else:
            return split_area(x, y, width, split_pos, first_half) + split_area(
                x, y + split_pos, width, height - split_pos, second_half
            )

    base_zones = split_area(0, 0, width, height, num_zones)
    return [
        {
            "id": i,
            "x": z["x"],
            "y": z["y"],
            "width": z["width"],
            "height": z["height"],
            "name": name,
            "colour": colour,
        }
        for i, (z, name, colour) in enumerate(
            zip(base_zones, names, colours)
        )
    ]

In [4]:
def generate_graph(num_horiz_roads, num_vert_roads, horiz_spacing, vert_spacing, ROAD_DENSITY):
    # Generate the street network
    street_graph = nx.Graph()

    # Create grid nodes
    for i in range(num_horiz_roads + 1):
        for j in range(num_vert_roads + 1):
            x = i * horiz_spacing
            y = j * vert_spacing
            node_id = f"{x},{y}"
            street_graph.add_node(node_id, x=x, y=y)

    # Connect horizontally
    for i in range(num_horiz_roads):
        for j in range(num_vert_roads + 1):
            x1 = i * horiz_spacing
            x2 = (i + 1) * horiz_spacing
            y = j * vert_spacing
            node1 = f"{x1},{y}"
            node2 = f"{x2},{y}"
            distance = horiz_spacing
            street_graph.add_edge(node1, node2, weight=distance)

    # Connect vertically
    for i in range(num_horiz_roads + 1):
        for j in range(num_vert_roads):
            x = i * horiz_spacing
            y1 = j * vert_spacing
            y2 = (j + 1) * vert_spacing
            node1 = f"{x},{y1}"
            node2 = f"{x},{y2}"
            distance = vert_spacing
            street_graph.add_edge(node1, node2, weight=distance)

    # Remove random edges
    edges = list(street_graph.edges())
    num_to_remove = int(len(edges) * (1-ROAD_DENSITY))
    edges_to_remove = random.sample(edges, num_to_remove)
    street_graph.remove_edges_from(edges_to_remove)

    return street_graph

In [ ]:
def generate_trips(num_trips, zones, points_in_zone, street_graph, time_per_unit):
    # Generate trips
    trips = []

    for i in range(num_trips):
        start_zone_index = random.randint(0, len(zones) - 1)
        start_zone = zones[start_zone_index]

        end_zone_index = random.randint(0, len(zones) - 1)
        end_zone = zones[end_zone_index]

        start_point = random.choice(points_in_zone[start_zone["id"]])
        end_point = random.choice(points_in_zone[end_zone["id"]])

        try:
            route = nx.shortest_path(street_graph, start_point["id"], end_point["id"], weight="weight")
        except nx.NetworkXNoPath:
            continue

        distance = nx.shortest_path_length(street_graph, start_point["id"], end_point["id"], weight="weight")
        travel_time = distance / time_per_unit

        trips.append(
            {
                "id": i,
                "start_zone_id": start_zone["id"],
                "start_zone_name": start_zone["name"],
                "end_zone_id": end_zone["id"],
                "end_zone_name": end_zone["name"],
                "travel_time": travel_time,
                "distance": distance,
                "real_start_point": dict(start_point),
                "real_end_point": dict(end_point),
                "estimated_start_point": None,
                "estimated_end_point": None,
                "route": route,
                "estimated_route": None,
            }
        )

    return trips

In [6]:
def generate_synthetic_city_data(
    road_density=0.85,
    num_zones=16,
    num_trips=20,
    time_per_unit=10,
    ):
    
    # Optimize dimensions to fit the grid best
    width = int(200 * (num_zones**0.5))
    height = int(200 * (num_zones**0.5))
    num_horiz_roads = 4 * num_zones
    num_vert_roads = 4 * num_zones

    width, height, num_horiz_roads, num_vert_roads, horiz_spacing, vert_spacing = find_best_grid_dimensions(
        width, height, num_horiz_roads, num_vert_roads
    )
    
    names, colours = generate_zone_labels(num_zones)

    zones = divide_and_assign_zone_data(width, height, num_zones, names, colours)
    
    street_graph = generate_graph(num_horiz_roads, num_vert_roads, horiz_spacing, vert_spacing, road_density)
    
    trips = generate_trips(num_trips, zones, get_street_points_by_zone(street_graph, zones), street_graph, time_per_unit) # type: ignore
    
    return zones, street_graph, trips, width, height
